<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Torrent_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install libtorrent gradio -q # Changed package name to libtorrent instead of python-libtorrent

import libtorrent as lt
import time
import gradio as gr

# ... (rest of your code remains the same)

def download_torrent(magnet_link):
    # Initialize session
    ses = lt.session()
    ses.listen_on(6881, 6891)
    params = {
        'save_path': '/content/',
        'storage_mode': lt.storage_mode_t(2)
    }

    # Add torrent
    handle = lt.add_magnet_uri(ses, magnet_link, params)
    ses.start_dht()

    # Wait for metadata
    print("Fetching metadata...")
    while not handle.has_metadata():
        time.sleep(1)

    torinfo = handle.get_torrent_info()
    filename = torinfo.name()
    file_path = f"/content/{filename}"

    # Start download
    print("Downloading...")
    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        print(f'Progress: {s.progress * 100:.2f}% | Speed: {s.download_rate / 1000:.1f} kB/s')
        time.sleep(5)

    return file_path

# Gradio interface
iface = gr.Interface(
    fn=download_torrent,
    inputs=gr.Textbox(label="Magnet Link", placeholder="Enter magnet link here..."),
    outputs=gr.File(label="Download File"),
    title="Torrent Downloader",
    description="Enter a magnet link to download content (Note: Colab sessions are temporary!)"
)

iface.launch(share=True)